In [ ]:
import numpy as np 
import pandas as pd
import xgboost as xgb
%run data_preprocessing.ipynb
%run create_X_Y.ipynb
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Input, Dropout
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.callbacks import EarlyStopping
from datetime import datetime, timedelta, date



In [ ]:
class post_model(hydropost):
    """
    Класс post_model наследует класс hydropost и служит для удобной работы с моделями.
    """
    def __init__(self, loc_id='3012', high_of_measurement=281):
        super().__init__(loc_id, high_of_measurement)

        
    def make_simple_formation(self):
        """
        Метод служит для быстрой предобработки данных по заранее определенному маршруту. После получения 
        прогноза, прогнозируемая величина должна быть обратно преобразованна в исходные еденицы измерения.
        """
        self.cut_level_nas(will_nas=8)
        self.interp_gaps(features=['water_temperature', 'snow_depth', 'ice_thickness'], fillwith=0, between=30)
        self.interp_gaps(features=set(self.hydroframe.columns.to_list())-{'min_level' , 'avg_level'},  between=20)
        self.diff_timeseries(features=['max_level'], add=True)
        self.log_timeseries(features=['max_level'])
        self.something_like_mrw(features=['max_level', 'maxairtemperature_mean', 'minairtemperature_mean', 'maxrelativehumidity_mean'])
        self.mean_timeseries(features=self.hydroframe.columns, scaling=True)
        
        
        
        
    def make_trains(self, frame=None,               # Фрейм, который следует сплитить. Если None cплитится hydroframe.
                    column_to_predict='max_level',  # Целевая переменная.
                    split_date='2014-01-01',        # Первая дата разбиения, до- трейн, после- тест.
                    split_date_2=None,              # Вторая дата разбиения, если указана, до- тест, после- валидационная.
                    day_forward=0,                  # На сколько Y сдвинут относительно X вперед (для XGB моделей).                
                    lags=None,                      # Добавляет лаггированые признаки к выборкам до lags дня.
                    ret=False,                      # Возвращать выборки или присвоить их атрибутам объекта.
                    cut_months=False):              # cut_months={'start':4, 'end':7}  Позволяет вырезать только необходимый временной промежуток по месяцам.
        """
        Метод разбивает hydroframe, или внешний frame на трейновую, тестовую, валидационнцю выборку с возможностью лаггирования признаков,
        обрезания месяцев, сдвига Y на day_forward дней вперед для бустинговых моделей. Выборки возвращаются, если ret=True, или присваиваются как атрибуты
        объекта.
        """
        
        if frame is None:
            frame=self.hydroframe 
        
    
        train = frame.loc[frame.index <= split_date].copy()
        try:
            test = frame.loc[(frame.index > split_date) & (frame.index < split_date_2)].copy()
        except TypeError:
            test = frame.loc[frame.index > split_date].copy()
            
        if split_date_2 != None:
            test2=frame.loc[frame.index > split_date_2].copy()
        listoffeatures= frame.columns.to_list()
        
        x_train=(train[listoffeatures].shift(day_forward)).iloc[(day_forward+1):]
        y_train=(train[column_to_predict]).iloc[(day_forward+1):]    
        x_test=(test[listoffeatures].shift(day_forward)).iloc[(day_forward+1):]
        y_test=(test[column_to_predict]).iloc[(day_forward+1):]
        
        if split_date_2 != None:
            x_test2=(test2[listoffeatures].shift(day_forward)).iloc[(day_forward+1):]
            y_test2=(test2[column_to_predict]).iloc[(day_forward+1):]
        if (lags is not None) and(lags!=0):
            try:
                lagged_x_train, lagged_x_test, lagged_x_test2= pd.DataFrame(index=x_train.index),pd.DataFrame(index=x_test.index), pd.DataFrame(index=x_test2.index)
            except UnboundLocalError: 
                lagged_x_train, lagged_x_test=pd.DataFrame(index=x_train.index),pd.DataFrame(index=x_test.index)
                
            for lag in range(1,lags+1):
                lagged_x_train=pd.merge(lagged_x_train, x_train.shift(lag).iloc[(lag+1):].add_prefix(('lag_'+str(lag))+'_'),left_index=True, right_index=True)        
                lagged_x_test=pd.merge(lagged_x_test, x_test.shift(lag).iloc[(lag+1):].add_prefix(('lag_'+str(lag))+'_'),left_index=True, right_index=True)
                if split_date_2 != None:
                    lagged_x_test2=pd.merge(lagged_x_test2, x_test2.shift(lag).iloc[(lag+1):].add_prefix(('lag_'+str(lag))+'_'),left_index=True, right_index=True)
            
            x_train=pd.merge(x_train, lagged_x_train, left_index=True, right_index=True, how='right')
            x_test=pd.merge(x_test, lagged_x_test, left_index=True, right_index=True,how='right')
            y_test=y_test.iloc[lags+1:]
            y_train=y_train.iloc[lags+1:]
            if split_date_2 != None:
                x_test2=pd.merge(x_test2, lagged_x_test2, left_index=True, right_index=True,how='right')
                y_test2=y_test2.iloc[lags+1:]         
        
        if cut_months!=False:
            if 'x_train' and 'y_train' and 'x_test' and 'y_test' and 'x_test2' and 'y_test2' in locals():
                x_train, y_train, x_test, y_test, x_test2, y_test2=x_train.loc[(x_train.index.month >= cut_months['start']) & (x_train.index.month <= cut_months['end'])],\
                                                                   y_train.loc[(y_train.index.month>= cut_months['start']) & (y_train.index.month <= cut_months['end'])],\
                                                                   x_test.loc[(x_test.index.month>= cut_months['start']) & (x_test.index.month <= cut_months['end'])],\
                                                                   y_test.loc[(y_test.index.month>= cut_months['start']) & (y_test.index.month <= cut_months['end'])],\
                                                                   x_test2.loc[(x_test2.index.month>= cut_months['start']) & (x_test2.index.month <= cut_months['end'])],\
                                                                   y_test2.loc[(y_test2.index.month>= cut_months['start']) & (y_test2.index.month <= cut_months['end'])]
                
            elif  'x_train' and 'y_train' and 'x_test' and 'y_test' in locals():
                x_train, y_train, x_test, y_test=x_train.loc[(x_train.index.month>= cut_months['start']) & (x_train.index.month<= cut_months['end'])],\
                                                 y_train.loc[(y_train.index.month>= cut_months['start']) & (y_train.index.month<= cut_months['end'])],\
                                                 x_test.loc[(x_test.index.month>= cut_months['start']) & (x_test.index.month<= cut_months['end'])],\
                                                 y_test.loc[(y_test.index.month>= cut_months['start']) & (y_test.index.month<= cut_months['end'])]                                                        
        
        if ret==False:
            if split_date_2 != None:
                self.x_train, self.y_train, self.x_test, self.y_test, self.x_test2, self.y_test2 =x_train, y_train, x_test, y_test,x_test2,y_test2
            elif split_date_2 == None:
                self.x_train, self.y_train, self.x_test, self.y_test=x_train, y_train, x_test, y_test                
        elif ret==True:
            if split_date_2 == None:
                return  x_train, y_train, x_test, y_test
            elif split_date_2 != None:
                return x_train, y_train, x_test, y_test,x_test2,y_test2
                
            

    

    def make_LSTM_model(self, cuted=False,
                        lr=0.001,
                        lag=14,
                        LSTM_units=34,             #Число элементов на LSTM-слое
                        batch_size=128,
                        dense_layers={1:50, 2: 50} #Число скрытых слоев и количество нейронов на каждом из слоев
                       ):
        """
        Метод обучает первую LSTM-модель (без разделения скрытых слоев) с заданными гиперпараметрами на трейновых, тестовых выборках, хранимых в атрибутах объекта.
        Готовая модель доступна по self.LSTM_model
        """
        self.LSTM_samples={}
        if cuted==False:

            
            x_train_np=self.x_train.reset_index(drop=True)
            x_train_np=self.x_train.to_numpy()
            x_test_np=self.x_test.reset_index(drop=True)
            x_test_np=self.x_test.to_numpy()
            x_test2_np=self.x_test2.reset_index(drop=True)
            x_test2_np=self.x_test2.to_numpy()

            self.LSTM_samples['Xtrain'],self.LSTM_samples['Ytrain']=create_X_Y(ts=x_train_np, lag=lag, n_ahead=7, target_index=0)  
            self.LSTM_samples['Xval'],self.LSTM_samples['Yval']=create_X_Y(ts=x_test_np, lag=lag, n_ahead=7, target_index=0)      
            self.LSTM_samples['Xtest'],self.LSTM_samples['Ytest']=create_X_Y(ts=x_test2_np, lag=lag, n_ahead=7, target_index=0)

        elif cuted==True:
            self.LSTM_samples['Xtrain'],\
            self.LSTM_samples['Ytrain'],\
            self.LSTM_samples['Xval'],\
            self.LSTM_samples['Yval'],\
            self.LSTM_samples['Xtest'],\
            self.LSTM_samples['Ytest']=create_X_Y_cuted(self.x_train,
                                                        self.y_train,
                                                        self.x_test,
                                                        self.y_test,
                                                        self.x_test2,
                                                        self.y_test2, lag=lag)
            
            
        n_steps=self.LSTM_samples['Xtrain'].shape[1]
        n_features=self.LSTM_samples['Xtrain'].shape[2]

        model = Sequential() 
        model.add(LSTM(units=LSTM_units, activation='tanh', return_sequences=False, input_shape=(n_steps, n_features )))
        for num in dense_layers:
            name = 'layer_dense_'+ str(num)
            model.add(Dense(dense_layers[num], activation='tanh', name=name))

            
        model.add(Dense(7,activation='tanh')) 
        model.compile(optimizer=Adam(learning_rate=lr), loss=MeanSquaredError()) 
    
        def trainCallback():
            return EarlyStopping(monitor='val_loss', patience=5, min_delta=0.00001, verbose=0)
        
        model.fit(x=self.LSTM_samples['Xtrain'], 
                        y=self.LSTM_samples['Ytrain'],
                        epochs=1000, 
                        validation_data=(self.LSTM_samples['Xval'],self.LSTM_samples['Yval']), 
                        shuffle=False,
                        batch_size=batch_size,
                        callbacks=[trainCallback()])
        self.LSTM_model=model
    

    

    def make_week_XGB_stack(self, 
                            cut_months=False,               
                            column_to_predict='max_level',  #Целевая переменная 
                            split_date='2014-01-01',        #Дата разбиения на трейновую и тестовую выборки
                            n_estimators=1000,
                            max_depth=6,
                            eta=0.1372,
                            subsample=0.9521413041539605,
                            colsample_bytree=1.0,
                            colsample_bylevel=0.72291,
                            min_child_weight= 0.24052,
                            reg_lambda=0.03559,
                            alpha=0.61582,
                            gamma=0
                           ):
        
        """
        Метод формирует выборки, обучает семь XGB моделей с заданными гиперпараметрами, сформированных по step-forward подходу, по одной для каждого дня прогноза.
        """
        if hasattr(self, 'week_XGB')==False:
            self.week_XGB = dict.fromkeys(['XGBmod_day_forward_'+ str(i) for i in range(1,8)])
        for num, model in enumerate(self.week_XGB):
            x_train, y_train, x_test, y_test=self.make_trains(column_to_predict=column_to_predict, split_date=split_date, day_forward=num+1, ret=True, cut_months=cut_months)
            self.week_XGB[model]=xgb.XGBRegressor( n_estimators=n_estimators, max_depth=max_depth, eta=eta, subsample=subsample, colsample_bytree=colsample_bytree, colsample_bylevel=colsample_bylevel, min_child_weight=min_child_weight,reg_lambda=reg_lambda,alpha=alpha,gamma=gamma)  
            self.week_XGB[model].fit(x_train, y_train,
            eval_set=[(x_train, y_train), (x_test, y_test)],
            early_stopping_rounds=50,
            verbose=False)   




        
    def make_models_prediction(self, 
                               actual_date='2017-12-20', 
                               column_to_predict='max_level',
                               LSTM_lag=13 ): #Лаг обязан быть тем же, с каким обучали модель
        """
        Метод позволяет формировать прогнозы XGB и LSTM моделей на актуальный день (если данные актуальны),
        или какой-либо день в прошлом, исходя из исторических данных. Возвращает словарь с прогнозами XGB,
        LSTM моделей и фактический уровень воды на дни предсказаний.
        """

        split_date=str(date.fromisoformat(actual_date)-timedelta(days=LSTM_lag+1))
        split_date_2=str(date.fromisoformat(actual_date)+timedelta(days=1))
        _, _, x_test, check,_,_=self.make_trains(split_date=split_date, split_date_2=split_date_2, day_forward=0,  column_to_predict=column_to_predict, ret=True)
        
        XGB_pred=np.array([])
        for model in self.week_XGB:
            XGB_pred=np.append(XGB_pred, float(self.week_XGB[model].predict(x_test.iloc[-1:] )))
        
        LSTM_input=x_test.reset_index(drop=True).to_numpy()
        nlag=LSTM_input.shape[0]
        nft=LSTM_input.shape[1]
        LSTM_pred=self.LSTM_model.predict(x_test.reset_index(drop=True).to_numpy().reshape(1, nlag, nft))
        LSTM_pred=np.array([float(x) for x in LSTM_pred])
        measured=self.hydroframe.loc[(self.hydroframe.index > actual_date) & (self.hydroframe.index <= str(date.fromisoformat(actual_date)+timedelta(days=7)))][column_to_predict].reset_index(drop=True).to_numpy()
        return {'XGB':XGB_pred, 'LSTM':LSTM_pred, 'measured':measured} 
        
        
    
    def make_stackframe(self, dateinterval={'start':'2010-01-01','end':'2016-01-01'}, 
                        LSTM_lag=13, #Лаг обязан быит таким же, с каким обучали модель 
                        ret=False):
        """
        Метод формирует выборку из предсказаний XGB, LSTM моделей  на временном интервале 
        для обучения ансамблирующих моделей. Готовый фрейм, помеченный днями дальности прогноза присваивается как 
        атрибут, либо возвращается. 
        """
        stackframe= self.hydroframe.loc[(self.hydroframe.index > dateinterval['start']) & (self.hydroframe.index < dateinterval['end'])].copy()
        for day in stackframe.index.strftime("%Y-%m-%d"):
            preddict=self.make_models_prediction(actual_date=day, LSTM_lag=LSTM_lag)
            for i in range(0,7):
                stackframe.loc[str(date.fromisoformat(day)+timedelta(i+1)), [str('XGB_'+str(i+1)+'_dayf')]] =float(preddict["XGB"][i])
                stackframe.loc[str(date.fromisoformat(day)+timedelta(i+1)), [str('LSTM_'+str(i+1)+'_dayf')]]=float(preddict["LSTM"][i])
        returnedframe=pd.DataFrame()
        for day in stackframe.index:
            for i in range(1,8):
                row=stackframe.loc[[day],['max_level','LSTM_'+str(i)+'_dayf', 'XGB_'+str(i)+'_dayf']]
                row.rename(columns={'LSTM_'+str(i)+'_dayf': 'LSTM', 'XGB_'+str(i)+'_dayf': 'XGB'}, inplace=True)
                row['dayf']=i
                returnedframe=pd.concat([returnedframe,row])
                returnedframe.dropna(inplace=True)
        if ret==False:
            self.stackframe=returnedframe
        elif ret==True:
            return returnedframe

        
        
    
    def make_stackmodel(self, 
                        column_to_predict='max_level',
                        model='ridge'):  #Так же доступны XGB, tree, lasso
        """
        Метод разворачивает признаки предсказаний XGB, LSTM моделей для семи дней в 
        пространство нескоррелированных признаков большей размерности с методом Kernel PCA, обучает на полученных метапризнаках один из выбранных базовых алгоритмов.
        """
        
        self.kpca_stack={}
        self.stackmodel={}
        for day in range(1,8):
            frame=self.stackframe.loc[self.stackframe['dayf']==day].drop(columns=['dayf'])
            #KERNEL PCA вот тут
            self.kpca_stack['dayf_'+str(day)] = KernelPCA( kernel='linear') #rbf
            decomposed = pd.concat([pd.DataFrame(self.kpca_stack['dayf_'+str(day)].fit_transform(frame.drop(columns=column_to_predict)), index=frame.index), frame[column_to_predict]], axis=1)
               

            if model=='ridge':
                self.stackmodel['dayf_'+str(day)]=RidgeCV(cv=50)
                self.stackmodel['dayf_'+str(day)].fit(X=decomposed.drop(columns='max_level'), y=decomposed['max_level'])
            elif model=='lasso':
                self.stackmodel['dayf_'+str(day)] = LassoCV(cv=50)
                self.stackmodel['dayf_'+str(day)].fit(X=decomposed.drop(columns='max_level'), y=decomposed['max_level'])
            
            elif model=='tree':
                self.stackmodel['dayf_'+str(day)]=DecisionTreeRegressor()
                self.stackmodel['dayf_'+str(day)].fit(X=decomposed.drop(columns='max_level'), y=decomposed['max_level'])
                
            elif model=='XGB':
                train, test =train_test_split(decomposed, test_size=0.3, random_state=42)   
                trains={'x_train':train.drop(columns=[column_to_predict]),
                    'y_train':train[column_to_predict],
                    'x_test':test.drop(columns=[column_to_predict]),
                    'y_test':test[column_to_predict]}
                                 
                self.stackmodel['dayf_'+str(day)]= xgb.XGBRegressor()
                self.stackmodel['dayf_'+str(day)].fit(trains['x_train'], trains['y_train'],
                                                eval_set=[(trains['x_train'], trains['y_train']), (trains['x_test'], trains['y_test'])],
                                                early_stopping_rounds=10,
                                                verbose=False) 
            


    def make_stack_prediction(self, actual_date='2017-12-20', column_to_predict='max_level', LSTM_lag=13):
        """
        Метод возвращает прогноз ансамбля моделей для соответствующего актуального дня.
        """
        m_prediction=self.make_models_prediction( actual_date=actual_date,column_to_predict=column_to_predict, LSTM_lag=LSTM_lag)
        
        frame=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in m_prediction.items() ])) #позволяет избежать ошибки разных длин 
        frame.index=pd.date_range(start=str(date.fromisoformat(actual_date)+timedelta(days=1)), end=str(date.fromisoformat(actual_date)+timedelta(days=7)))
        stack_prediction=np.array([])
        for day in range(1,8):
            transformed=self.kpca_stack['dayf_'+str(day)].transform(frame.iloc[[day-1]].drop(columns='measured'))
            stack_prediction=np.append(stack_prediction, self.stackmodel['dayf_'+str(day)].predict(transformed))
        frame['stack_pred']=stack_prediction
        
        for col in frame.columns:
            frame[col]=self.mean_timeseries(inverse=frame[col], target='max_level') 
            frame[col]=self.something_like_mrw(inv=frame[col], target='max_level' )
            frame[col]=np.exp(frame[col])
        return frame 

        
        
        
    def make_LSTM2_model(self, cuted=True, lr=0.00122, lag=14, LSTM_units=34, dense_small=10, dense_small1=10, batch_size=128):
        """
        Вторая LSTM-модель - модель, архитектура которой дала лучшие результаты прогнозирования на недельном интервале
        """ 
        self.LSTM_samples={}
        if cuted==False:

            
            x_train_np=self.x_train.reset_index(drop=True)
            x_train_np=self.x_train.to_numpy()
            x_test_np=self.x_test.reset_index(drop=True)
            x_test_np=self.x_test.to_numpy()
            x_test2_np=self.x_test2.reset_index(drop=True)
            x_test2_np=self.x_test2.to_numpy()

            self.LSTM_samples['Xtrain'],self.LSTM_samples['Ytrain']=create_X_Y(ts=x_train_np, lag=lag, n_ahead=7, target_index=0) #Обучающий набор 
            self.LSTM_samples['Xval'],self.LSTM_samples['Yval']=create_X_Y(ts=x_test_np, lag=lag, n_ahead=7, target_index=0)      #Валидационный набор
            self.LSTM_samples['Xtest'],self.LSTM_samples['Ytest']=create_X_Y(ts=x_test2_np, lag=lag, n_ahead=7, target_index=0)

        elif cuted==True:
            self.LSTM_samples['Xtrain'],\
            self.LSTM_samples['Ytrain'],\
            self.LSTM_samples['Xval'],\
            self.LSTM_samples['Yval'],\
            self.LSTM_samples['Xtest'],\
            self.LSTM_samples['Ytest']=create_X_Y_cuted(self.x_train,
                                                        self.y_train,
                                                        self.x_test,
                                                        self.y_test,
                                                        self.x_test2,
                                                        self.y_test2, lag=lag)
            
            
        n_steps=self.LSTM_samples['Xtrain'].shape[1]
        n_features=self.LSTM_samples['Xtrain'].shape[2]
        
        inputs = tensorflow.keras.layers.Input(shape=(n_steps, n_features ))
        lstm_big = LSTM(units=LSTM_units, activation='tanh', return_sequences=False)(inputs)
        
        
        dense_small_1=Dense(dense_small, activation='tanh')(lstm_big)
        dense_small_2=Dense(dense_small, activation='tanh')(lstm_big)
        dense_small_3=Dense(dense_small, activation='tanh')(lstm_big)
        dense_small_4=Dense(dense_small, activation='tanh')(lstm_big)
        dense_small_5=Dense(dense_small, activation='tanh')(lstm_big)
        dense_small_6=Dense(dense_small, activation='tanh')(lstm_big)
        dense_small_7=Dense(dense_small, activation='tanh')(lstm_big)
        
        drop_1=Dropout(0.5, seed=1)(dense_small_1)
        drop_2=Dropout(0.5, seed=1)(dense_small_2)
        drop_3=Dropout(0.5, seed=1)(dense_small_3)
        drop_4=Dropout(0.5, seed=1)(dense_small_4)
        drop_5=Dropout(0.5, seed=1)(dense_small_5)
        drop_6=Dropout(0.5, seed=1)(dense_small_6)
        drop_7=Dropout(0.5, seed=1)(dense_small_7)
        
        
        dense_small1_1=Dense(dense_small1, activation='tanh')(dense_small_1)
        dense_small1_2=Dense(dense_small1, activation='tanh')(dense_small_2)
        dense_small1_3=Dense(dense_small1, activation='tanh')(dense_small_3)
        dense_small1_4=Dense(dense_small1, activation='tanh')(dense_small_4)
        dense_small1_5=Dense(dense_small1, activation='tanh')(dense_small_5)
        dense_small1_6=Dense(dense_small1, activation='tanh')(dense_small_6)
        dense_small1_7=Dense(dense_small1, activation='tanh')(dense_small_7)
        
        
        drop1_1=Dropout(0.5, seed=1)(dense_small1_1)
        drop1_2=Dropout(0.5, seed=1)(dense_small1_2)
        drop1_3=Dropout(0.5, seed=1)(dense_small1_3)
        drop1_4=Dropout(0.5, seed=1)(dense_small1_4)
        drop1_5=Dropout(0.5, seed=1)(dense_small1_5)
        drop1_6=Dropout(0.5, seed=1)(dense_small1_6)
        drop1_7=Dropout(0.5, seed=1)(dense_small1_7)
        
        
        out_1=Dense(1)(drop1_1) #(dense_small1_1)
        out_2=Dense(1)(drop1_2)
        out_3=Dense(1)(drop1_3)
        out_4=Dense(1)(drop1_4)
        out_5=Dense(1)(drop1_5)
        out_6=Dense(1)(drop1_6)
        out_7=Dense(1)(drop1_7)
        
        

        model = tensorflow.keras.Model(inputs=inputs, outputs=[out_1,out_2,out_3,out_4,out_5,out_6,out_7])
        model.compile(optimizer=tensorflow.keras.optimizers.Adam(learning_rate=lr), loss="mse")
        model.summary()
        
        
        model.compile(optimizer=Adam(learning_rate=lr), loss=MeanSquaredError()) 
            
        def trainCallback():
            return EarlyStopping(monitor='val_loss', patience=10, min_delta=0.00001, verbose=0)
        
        self.LSTM_history=model.fit(x=self.LSTM_samples['Xtrain'], 
                        y=self.LSTM_samples['Ytrain'],
                        epochs=1000, 
                        validation_data=(self.LSTM_samples['Xval'],self.LSTM_samples['Yval']), 
                        shuffle=False,
                        batch_size=batch_size,
                        callbacks=[trainCallback()])
        self.LSTM_model=model 
        
        
